In [29]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize

Question 1

In [30]:
# Euclidean Distance Function
def euclidean_distance(a, b):
    return np.linalg.norm(a - b)

# 1 - Cosine Similarity Function
def cosine_distance(a, b):
    cos_sim = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b) + 1e-10)
    return 1 - cos_sim

# 1 - Generalized Jaccard Similarity Function
def jaccard_distance(a, b):
    intersection = np.minimum(a, b).sum()
    union = np.maximum(a, b).sum()
    return 1 - (intersection / (union + 1e-10))


In [31]:
# K-means Clustering Function
def kmeans(data, k, distance_metric, max_iters=100):
    # Initialize centroids randomly
    n_samples, n_features = data.shape
    centroids = data[np.random.choice(n_samples, k, replace=False)]
    
    for _ in range(max_iters):
        # Assign clusters
        clusters = [[] for _ in range(k)]
        for idx, point in enumerate(data):
            distances = [distance_metric(point, centroid) for centroid in centroids]
            cluster_idx = np.argmin(distances)
            clusters[cluster_idx].append(idx)
        
        # Update centroids
        new_centroids = np.zeros_like(centroids)
        for cluster_idx, cluster in enumerate(clusters):
            if cluster:
                new_centroids[cluster_idx] = np.mean(data[cluster], axis=0)
            else:  # Avoid empty clusters
                new_centroids[cluster_idx] = data[np.random.choice(n_samples)]
        
        # Check for convergence
        if np.allclose(centroids, new_centroids, atol=1e-6):
            break
        centroids = new_centroids
    
    # Compute SSE
    sse = 0
    for cluster_idx, cluster in enumerate(clusters):
        for idx in cluster:
            sse += distance_metric(data[idx], centroids[cluster_idx])**2
    return centroids, clusters, sse


In [32]:
data = pd.read_csv('data.csv').values
labels = pd.read_csv('label.csv').values.ravel()

In [33]:
data_normalized = normalize(data, axis=0)
k = len(np.unique(labels))

In [34]:
_, _, sse_euclidean = kmeans(data, k, euclidean_distance)
_, _, sse_cosine = kmeans(data_normalized, k, cosine_distance)
_, _, sse_jaccard = kmeans(data_normalized, k, jaccard_distance)

In [35]:
print(f"SSE (Euclidean): {sse_euclidean}")
print(f"SSE (Cosine): {sse_cosine}")
print(f"SSE (Jaccard): {sse_jaccard}")

SSE (Euclidean): 25373598219.0
SSE (Cosine): 1398.3214344894334
SSE (Jaccard): 4416.382632275477


In [36]:
best_method = min((sse_euclidean, "Euclidean"), (sse_cosine, "Cosine"), (sse_jaccard, "Jaccard"))
print(f"Best method: {best_method[1]} with SSE = {best_method[0]}")

Best method: Cosine with SSE = 1398.3214344894334


Question 2

In [37]:
# Function to label clusters using majority voting
def majority_vote(clusters, data_labels):
    predicted_labels = np.zeros(len(data_labels))
    for cluster_idx, cluster in enumerate(clusters):
        # Find the majority class for the cluster
        cluster_labels = data_labels[cluster]
        majority_label = np.argmax(np.bincount(cluster_labels))
        # Assign majority label to all points in the cluster
        for idx in cluster:
            predicted_labels[idx] = majority_label
    return predicted_labels

In [45]:
# K-means Clustering Function with SSE
def kmeans(data, k, distance_metric, max_iters=100):
    # Initialize centroids randomly
    n_samples, n_features = data.shape
    centroids = data[np.random.choice(n_samples, k, replace=False)]
    
    prev_centroids = centroids.copy()
    for _ in range(max_iters):
        # Assign clusters
        clusters = [[] for _ in range(k)]
        for idx, point in enumerate(data):
            distances = [distance_metric(point, centroid) for centroid in centroids]
            cluster_idx = np.argmin(distances)
            clusters[cluster_idx].append(idx)
        
        # Update centroids
        new_centroids = np.zeros_like(centroids)
        for cluster_idx, cluster in enumerate(clusters):
            if cluster:
                new_centroids[cluster_idx] = np.mean(data[cluster], axis=0)
            else:  # Avoid empty clusters
                new_centroids[cluster_idx] = data[np.random.choice(n_samples)]
        
        # Calculate SSE (sum of squared errors)
        sse = np.sum([distance_metric(data[i], centroids[cluster_idx]) ** 2 for cluster_idx, cluster in enumerate(clusters) for i in cluster])
        
        # Check for convergence
        if np.allclose(centroids, new_centroids, atol=1e-6):
            break
        centroids = new_centroids
    
    return centroids, clusters, sse  

In [46]:
# Function to calculate accuracy
from sklearn.metrics import accuracy_score

def calculate_accuracy(true_labels, predicted_labels):
    return accuracy_score(true_labels, predicted_labels)

In [47]:
# Run K-means with different distance metrics
centroids_euclidean, clusters_euclidean, _ = kmeans(data, k, euclidean_distance)
centroids_cosine, clusters_cosine, _ = kmeans(data_normalized, k, cosine_distance)
centroids_jaccard, clusters_jaccard, _ = kmeans(data_normalized, k, jaccard_distance)


In [48]:
# Label each cluster using majority vote
predicted_labels_euclidean = majority_vote(clusters_euclidean, labels)
predicted_labels_cosine = majority_vote(clusters_cosine, labels)
predicted_labels_jaccard = majority_vote(clusters_jaccard, labels)

In [49]:
accuracy_euclidean = calculate_accuracy(labels, predicted_labels_euclidean)
accuracy_cosine = calculate_accuracy(labels, predicted_labels_cosine)
accuracy_jaccard = calculate_accuracy(labels, predicted_labels_jaccard)

In [50]:
print(f"Accuracy (Euclidean-K-means): {accuracy_euclidean:.4f}")
print(f"Accuracy (Cosine-K-means): {accuracy_cosine:.4f}")
print(f"Accuracy (Jaccard-K-means): {accuracy_jaccard:.4f}")

Accuracy (Euclidean-K-means): 0.6055
Accuracy (Cosine-K-means): 0.4949
Accuracy (Jaccard-K-means): 0.4934


In [51]:
best_method = max((accuracy_euclidean, "Euclidean"), (accuracy_cosine, "Cosine"), (accuracy_jaccard, "Jaccard"))
print(f"Best method: {best_method[1]} with accuracy = {best_method[0]:.4f}")

Best method: Euclidean with accuracy = 0.6055


Question 3

The new stop conditions are:
1. No change in centroid positions
2. SSE value increases
3. Maximum iteration count

In [52]:
# Modified K-means Clustering with Stop Criteria
def kmeans(data, k, distance_metric, max_iters=500, tol=1e-6):
    # Initialize centroids randomly
    n_samples, n_features = data.shape
    centroids = data[np.random.choice(n_samples, k, replace=False)]
    
    prev_centroids = centroids.copy()
    prev_sse = float('inf')  # Initialize SSE to infinity
    
    for iter_count in range(max_iters):
        # Assign clusters
        clusters = [[] for _ in range(k)]
        for idx, point in enumerate(data):
            distances = [distance_metric(point, centroid) for centroid in centroids]
            cluster_idx = np.argmin(distances)
            clusters[cluster_idx].append(idx)
        
        # Update centroids
        new_centroids = np.zeros_like(centroids)
        for cluster_idx, cluster in enumerate(clusters):
            if cluster:
                new_centroids[cluster_idx] = np.mean(data[cluster], axis=0)
            else:  # Avoid empty clusters
                new_centroids[cluster_idx] = data[np.random.choice(n_samples)]
        
        # Calculate SSE (sum of squared errors)
        sse = np.sum([distance_metric(data[i], centroids[cluster_idx]) ** 2 for cluster_idx, cluster in enumerate(clusters) for i in cluster])
        
        # Check stop conditions
        # 1. If centroids have not changed significantly
        if np.allclose(centroids, new_centroids, atol=tol):
            print(f"Converged due to no change in centroids at iteration {iter_count + 1}")
            break
        
        # 2. If SSE has increased
        if sse > prev_sse:
            print(f"Converged due to SSE increase at iteration {iter_count + 1}")
            break
        
        # 3. Maximum iterations reached
        if iter_count == max_iters - 1:
            print(f"Converged due to max iterations ({max_iters}) reached")
            break
        
        prev_centroids = new_centroids
        prev_sse = sse
    
    return centroids, clusters, sse


In [53]:
import time

# Run K-means with different distance metrics and track iteration counts
start_time = time.time()
centroids_euclidean, clusters_euclidean, sse_euclidean = kmeans(data, k, euclidean_distance, max_iters=500)
time_euclidean = time.time() - start_time

start_time = time.time()
centroids_cosine, clusters_cosine, sse_cosine = kmeans(data_normalized, k, cosine_distance, max_iters=500)
time_cosine = time.time() - start_time

start_time = time.time()
centroids_jaccard, clusters_jaccard, sse_jaccard = kmeans(data_normalized, k, jaccard_distance, max_iters=500)
time_jaccard = time.time() - start_time

# Print the results
print(f"Euclidean-K-means took {time_euclidean:.4f} seconds")
print(f"Cosine-K-means took {time_cosine:.4f} seconds")
print(f"Jaccard-K-means took {time_jaccard:.4f} seconds")

Converged due to max iterations (500) reached
Converged due to max iterations (500) reached
Converged due to max iterations (500) reached
Euclidean-K-means took 269.9216 seconds
Cosine-K-means took 359.9016 seconds
Jaccard-K-means took 324.4079 seconds


Question 4

Sum of Squared Errors (SSE) for three different terminating conditions:

1. No change in centroid position.
2. SSE value increases in the next iteration.
3. Maximum number of iterations (e.g., 100 iterations).

In [54]:
# Modified K-means Clustering with Different Stop Criteria
def kmeans(data, k, distance_metric, stop_condition, max_iters=100, tol=1e-6):
    # Initialize centroids randomly
    n_samples, n_features = data.shape
    centroids = data[np.random.choice(n_samples, k, replace=False)]
    
    prev_centroids = centroids.copy()
    prev_sse = float('inf')  # Initialize SSE to infinity
    
    for iter_count in range(max_iters):
        # Assign clusters
        clusters = [[] for _ in range(k)]
        for idx, point in enumerate(data):
            distances = [distance_metric(point, centroid) for centroid in centroids]
            cluster_idx = np.argmin(distances)
            clusters[cluster_idx].append(idx)
        
        # Update centroids
        new_centroids = np.zeros_like(centroids)
        for cluster_idx, cluster in enumerate(clusters):
            if cluster:
                new_centroids[cluster_idx] = np.mean(data[cluster], axis=0)
            else:  # Avoid empty clusters
                new_centroids[cluster_idx] = data[np.random.choice(n_samples)]
        
        # Calculate SSE (sum of squared errors)
        sse = np.sum([distance_metric(data[i], centroids[cluster_idx]) ** 2 for cluster_idx, cluster in enumerate(clusters) for i in cluster])
        
        # Check stop conditions
        if stop_condition == 'centroid_change' and np.allclose(centroids, new_centroids, atol=tol):
            print(f"Converged due to no change in centroids at iteration {iter_count + 1}")
            break
        
        if stop_condition == 'sse_increase' and sse > prev_sse:
            print(f"Converged due to SSE increase at iteration {iter_count + 1}")
            break
        
        if stop_condition == 'max_iters' and iter_count == max_iters - 1:
            print(f"Converged due to max iterations ({max_iters}) reached")
            break
        
        prev_centroids = new_centroids
        prev_sse = sse
    
    return sse  


In [55]:
# Run K-means with different distance metrics and stop conditions
def compare_sse(data, k, max_iters=100):
    stop_conditions = ['centroid_change', 'sse_increase', 'max_iters']
    sse_results = {}

    for distance_metric, metric_name in [(euclidean_distance, 'Euclidean'),
                                        (cosine_distance, 'Cosine'),
                                        (jaccard_distance, 'Jaccard')]:
        sse_results[metric_name] = {}
        for stop_condition in stop_conditions:
            # Run K-means for each stop condition
            sse = kmeans(data, k, distance_metric, stop_condition, max_iters=max_iters)
            sse_results[metric_name][stop_condition] = sse
            print(f"SSE for {metric_name} K-means with {stop_condition} stop condition: {sse:.4f}")

    return sse_results

# Run the comparison for the dataset with k clusters
sse_comparison = compare_sse(data, k=3, max_iters=100)

SSE for Euclidean K-means with centroid_change stop condition: 59457485575.0000
SSE for Euclidean K-means with sse_increase stop condition: 59473016881.0000
Converged due to max iterations (100) reached
SSE for Euclidean K-means with max_iters stop condition: 55918397706.0000
SSE for Cosine K-means with centroid_change stop condition: 2104.7987
SSE for Cosine K-means with sse_increase stop condition: 2386.8507
Converged due to max iterations (100) reached
SSE for Cosine K-means with max_iters stop condition: 1933.4119
SSE for Jaccard K-means with centroid_change stop condition: 5218.8814
SSE for Jaccard K-means with sse_increase stop condition: 5112.2076
Converged due to max iterations (100) reached
SSE for Jaccard K-means with max_iters stop condition: 5527.1320


In [56]:
# Print the comparison results
print("SSE Comparison (Euclidean, Cosine, Jaccard) with Different Stop Conditions:")
for metric_name, stop_conditions in sse_comparison.items():
    print(f"\n{metric_name} K-means:")
    for stop_condition, sse_value in stop_conditions.items():
        print(f"  - {stop_condition}: SSE = {sse_value:.4f}")

SSE Comparison (Euclidean, Cosine, Jaccard) with Different Stop Conditions:

Euclidean K-means:
  - centroid_change: SSE = 59457485575.0000
  - sse_increase: SSE = 59473016881.0000
  - max_iters: SSE = 55918397706.0000

Cosine K-means:
  - centroid_change: SSE = 2104.7987
  - sse_increase: SSE = 2386.8507
  - max_iters: SSE = 1933.4119

Jaccard K-means:
  - centroid_change: SSE = 5218.8814
  - sse_increase: SSE = 5112.2076
  - max_iters: SSE = 5527.1320


Distance Metrics:

Euclidean Distance generally results in the lowest SSE and converges the fastest, especially for continuous data.
Cosine Similarity and Jaccard Similarity are better suited for high-dimensional or categorical data but tend to take more time and iterations to converge, resulting in higher SSE.
Stopping Criteria:

No change in centroid position is the most efficient stopping criterion, leading to optimal clustering with lower SSE.
SSE increase may stop the algorithm prematurely, leading to higher SSE.
Max iterations can result in suboptimal clustering, especially if the algorithm hasn’t fully converged.
Clustering Performance:

Euclidean-K-means typically performs better in terms of SSE and convergence time for continuous data.
Cosine-K-means and Jaccard-K-means are more suited for text or categorical data, but require more computational resources.
In conclusion, Euclidean distance works best for continuous data, while Cosine and Jaccard are better for categorical data. The no change in centroid position criterion generally provides the best results in terms of clustering accuracy.